In [0]:
import requests
import json
import time
from pyspark.sql.types import StructType, StringType, FloatType, IntegerType, TimestampType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json, current_timestamp, to_json, struct

# API configuration
API_URL = "http://10.223.106.19:8000"
MAX_RETRIES = 2
RETRY_DELAY = 2  # seconds

# Retrieve API key from Databricks Secret
try:
    API_KEY = dbutils.secrets.get(scope="dev_secret_scope", key="fast-api-key")
    print("API key retrieved from secrets")
except Exception as e:
    print(f"Error retrieving API key: {e}")
    raise Exception(f"Failed to retrieve API key: {e}")

# Get the workflow name from the notebook context or widget
workflow_name = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().get("jobName")

if workflow_name:
    workflow_name = str(workflow_name)
    if workflow_name.startswith("Some(") and workflow_name.endswith(")"):
        workflow_name = workflow_name[5:-1]  # Extract the inner value

script_name = workflow_name
print(f"Script name: {script_name}")
all_values = []
widget_values = dbutils.widgets.getAll()
print(widget_values)
if 'env' in widget_values:
    all_values.append(widget_values['env'])

# Add other parameters
for key, value in widget_values.items():
    if key != 'env' and value:  # Skip 'env' as it is already added
        all_values.append(value)

parameters_str = " ".join(all_values)

print(f"Combined parameter values: '{parameters_str}'")

print(f"Running script: {script_name}")

# Check if API is accessible
def check_api_health():
    try:
        response = requests.get(
            f"{API_URL}/health",
            headers={"X-API-Key": API_KEY},
            timeout=10
        )
        if response.status_code == 200:
            print("API is accessible!")
            return True
        else:
            error_detail = response.json().get("detail", response.text) if response.content else "No response content"
            print(f"API returned status code: {response.status_code}, Detail: {error_detail}")
            return False
    except Exception as e:
        print(f"Error connecting to API: {e}")
        return False

# Run a script with parameters and get results with retry logic
def run_script(script_name, parameters_str):
    retries = 0
    while retries < MAX_RETRIES:
        try:
            payload = {
                "script_name": script_name
            }
            
            if parameters_str:
                payload["parameters"] = parameters_str
                
            response = requests.post(
                f"{API_URL}/run-script",
                json=payload,
                headers={"X-API-Key": API_KEY},
                timeout=300  
            )
            
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 429:  # Rate limit exceeded
                retry_after = int(response.headers.get('Retry-After', RETRY_DELAY))
                error_detail = response.json().get("detail", "Rate limit exceeded") if response.content else "No response content"
                print(f"Rate limit exceeded: {error_detail}. Retrying in {retry_after} seconds...")
                time.sleep(retry_after)
                retries += 1
                continue
            else:
                error_detail = response.json().get("detail", response.text) if response.content else "No response content"
                raise Exception(f"API error: Status code {response.status_code}, Detail: {error_detail}")
        except Exception as e:
            print(f"Error running script: {e}")
            retries += 1
            if retries < MAX_RETRIES:
                print(f"Retrying ({retries}/{MAX_RETRIES})...")
                time.sleep(RETRY_DELAY)
            continue
    
    raise Exception(f"Maximum retries ({MAX_RETRIES}) exceeded. Unable to execute script.")

def prepare_data_for_storage(json_data):
    if not json_data:
        print("No data to store")
        return None
    
    spark = SparkSession.builder.getOrCreate()
    
    # Convert the entire JSON output to a string
    text_value = json.dumps(json_data)
    
    # Create a DataFrame with text_value and updated_date columns
    data = [(text_value,)]
    df = spark.createDataFrame(data, ["text_value"])
    
    # Add updated_date column with current timestamp
    df = df.withColumn("updated_date", current_timestamp())
    
    return df

if not script_name:
    raise Exception("No script name provided. Please provide a script_name parameter.")

# Check API health
if not check_api_health():
    raise Exception("Cannot run script. API is not accessible.")

# Run the script with parameters
print(f"Running script: {script_name} with parameters: {parameters_str}")
result = run_script(script_name, parameters_str)

# Check if the script execution was successful
if not result.get("success", False):
    error_msg = result.get("output", "No error message provided by API")
    raise Exception(f"Script execution failed: {error_msg}")

# If execution is successful, proceed with processing and storing results
print(f"\nScript execution successful: {result['success']}")

# Log any errors for reference (non-fatal stderr)
if result.get("error"):
    print(f"\nExecution stderr (logged for reference):\n{result['error']}")

# Convert to simple DataFrame with text_value and updated_date
storage_df = prepare_data_for_storage(result)
if storage_df is not None:
    print("\nData prepared for storage:")
    display(storage_df)
    # Save to a Delta table
    table_name = f"{script_name.replace('.', '_').replace('-', '_')}"
    storage_df.write.format("delta").mode("append").saveAsTable(table_name)
    print(f"Results saved to table: {table_name}")
else:
    raise Exception("Failed to prepare data for storage.")

API key retrieved from secrets
Script name: None
Combined parameter values: ''
Running script: None
API is accessible!
Running script: None with parameters: 
Error running script: API error: Status code 404, Detail: Script not found: None
Retrying (1/2)...
Error running script: API error: Status code 404, Detail: Script not found: None


---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
File <command-4605066948949457>, line 214
    212 # Run the script with parameters
    213 print(f"Running script: {script_name} with parameters: {parameters_str}")
--> 214 result = run_script(script_name, parameters_str)
    216 # Check if the script execution was successful
    217 if not result.get("success", False):

File <command-4605066948949457>, line 131, in run_script(script_name, parameters_str)
    128             time.sleep(RETRY_DELAY)
    129         continue
--> 131 raise Exception(f"Maximum retries ({MAX_RETRIES}) exceeded. Unable to execute script.")

Exception: Maximum retries (2) exceeded. Unable to execute script.